In [1]:
TOTAL_CHAR = 2**16


# derivative first half (log P(s) / alpha)
import numpy as np

import sys
import pickle
import math
import numpy as np
import pdb

import italian_vocab
import chinese

# n = 4   # number of letters as context
MAX_N = 4


class Blending_with_linear_param:
    def __init__(self):
        return None
    
    def derivative_of_P_to_alpha(self, alpha_matrix, beta_matrix, model, prefix, x):
        
        n = len(prefix)
        alpha, beta = self.compute_alpha_beta(alpha_matrix, beta_matrix, prefix)
        if prefix in model:
            ms = sum(model[prefix].values())
            us = len(model[prefix])
            if x in model[prefix]:
                ms_x = model[prefix][x]
            else:
                ms_x = 0
        else:
            ms = 0
            us = 0
            ms_x = 0
            
        eq1 = (beta-ms_x) / (ms+alpha)**2
        
        if len(prefix) == 0:
            eq2 = (ms-us*beta) / (ms+alpha)**2 / TOTAL_CHAR
            eq3 = 0
        else:
            next_prefix = prefix[1:]
            eq2 = (ms-us*beta) / (ms+alpha)**2 * self.blend(alpha_matrix, beta_matrix, next_prefix, x, model)
            eq3 = (alpha + us*beta) / (ms+alpha) * self.derivative_of_P_to_beta(alpha_matrix, beta_matrix, model, next_prefix, x)
        
        return eq1 + eq2 + eq3
            
    def derivative_of_P_to_beta(self, alpha_matrix, beta_matrix, model, prefix, x):
        
        n = len(prefix)
        alpha, beta = self.compute_alpha_beta(alpha_matrix, beta_matrix, prefix)
        if prefix in model:
            ms = sum(model[prefix].values())
            us = len(model[prefix])
            if x in model[prefix]:
                ms_x = model[prefix][x]
            else:
                ms_x = 0
        else:
            ms = 0
            us = 0
            ms_x = 0
            
        eq1 = -1 / (ms+alpha)
        if len(prefix) == 0:
            eq2 = (ms-us*beta) / (ms+alpha)**2 / TOTAL_CHAR
            eq3 = 0
        else:
            next_prefix = prefix[1:]
            eq2 = us / (ms+alpha) * self.blend(alpha_matrix, beta_matrix, next_prefix, x, model, n)
            eq3 = (alpha + us*beta) / (ms+alpha) * self.derivative_of_P_to_alpha(alpha_matrix, beta_matrix, model, next_prefix, x)
        
        return eq1 + eq2 + eq3
            

    def update_param(self, alpha_matrix, beta_matrix, prefix, x, model):
        learning_rate = 0.003
        n = len(prefix)
        # add a computation for alpha beta
        param_derivative_mapping = [1, len(set(prefix))]
        
        
        for ind, alpha_derivative in zip(range(len(alpha_matrix[n])), param_derivative_mapping):
            derivative_alpha = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_alpha(alpha_matrix, beta_matrix, model, prefix, x) * alpha_derivative
            alpha_matrix[n][ind] += learning_rate * derivative_alpha
            
        for ind, beta_derivative in zip(range(len(beta_matrix[n])), param_derivative_mapping):
            derivative_beta = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_beta(alpha_matrix, beta_matrix, model, prefix, x) * beta_derivative
            beta_matrix[n][ind] += learning_rate * derivative_beta
            
        beta_matrix[n][0] = max(0, min(1, beta_matrix[n][0]))
        if beta_matrix[n][0] + n * beta_matrix[n][1] > 1:
            beta_matrix[n][1] = (1 - beta_matrix[n][0]) / n
        elif beta_matrix[n][0] + n * beta_matrix[n][1] < 0:
            beta_matrix[n][1] = -beta_matrix[n][0] / n
            

    def compute_alpha_beta(self, alpha_matrix, beta_matrix, prefix):
        n = len(prefix)
        param_derivative_mapping = [1, len(set(prefix))]
        alpha = np.dot(alpha_matrix[n], param_derivative_mapping)
        beta = np.dot(beta_matrix[n], param_derivative_mapping)
        
        #beta = max(0, min(1, beta))
        alpha = max(-beta, alpha)
        
        
        return (alpha, beta)

    def build_model_with_train(self, alpha_matrix, beta_matrix, word_file, n):
        """
        Build n-gram model of the words in  words_lang.txt
        """
        n_grams = {}
        # read in all n+1 grams
        n_plus_1_gram_counts = {}
        with open(word_file, "r") as f:
            for word in f:
                # update n-gram
                if not word or word[0] == "#" or " " in word:
                    continue
                word = "^" * n + word.strip() + "$"
                for i in range(len(word) - n):
                    n_plus_1_gram = word[i:i + n + 1]
                        
                    for N in range(n, -1, -1):
                        x = n_plus_1_gram[-1]
                        next_n = n_plus_1_gram[:-1]
                        
                        if next_n not in n_grams:
                            n_grams[next_n] = {}
                            
                        if x not in n_grams[next_n]:
                            n_grams[next_n][x] = 1
                        else:
                            n_grams[next_n][x] += 1
                            
                        n_plus_1_gram = n_plus_1_gram[1:]
                        
                # update alpha and beta
                for i in range(len(word) - n):
                    n_plus_1_gram = word[i:i + n + 1]
                        
                    for N in range(n, -1, -1):
                        x = n_plus_1_gram[-1]
                        next_n = n_plus_1_gram[:-1]
                        
                        self.update_param(alpha_matrix, beta_matrix, next_n, x, n_grams)
                            
                        n_plus_1_gram = n_plus_1_gram[1:]

            return n_grams
    
    def blend(self, alpha_matrix, beta_matrix, prefix, x, model, n=4):
        prob = 0
        
        # compute alpha and beta
        alpha, beta = self.compute_alpha_beta(alpha_matrix, beta_matrix, prefix)
        possible_x = TOTAL_CHAR

        # first half of algorithm
        if prefix in model:
            base = (len(model[prefix])*beta + alpha) / (sum(model[prefix].values()) + alpha)
            if x in model[prefix]:
                prob += (model[prefix][x] - beta) / (sum(model[prefix].values()) + alpha)
        else:
            base = (0 + alpha) / (0 + alpha)
            
        if len(prefix) == 0:
            prob += base / possible_x
        else:
            prob += base * self.blend(alpha_matrix, beta_matrix, prefix[1:], x, model, n)
            
        return prob


    def word_prob_blend(self, word, model, n=4):
        word = "^" * n + word + "$"
        pos = n  # char after n ^
        log_likelihood = 0
        # print ("   ", end="")
        while pos < len(word):
            prefix = word[pos - n:pos]
            prob = self.blend(prefix, word[pos], model)
            log_likelihood += math.log (prob)
            pos += 1


        return log_likelihood / (len(word) - 1)  # didn't guess "^"



In [2]:
# trainer with 2d matrix for d and f

class Blending_with_2d_matrix(Blending_with_linear_param):
    def init(self):
        return None
    
    def update_param(self, alpha_matrix, beta_matrix, prefix, x, model):
        max_f = len(alpha_matrix[0])
        learning_rate = 0.003
        d = len(prefix)
        n = d
        f = len(set(prefix))
        
        if f > max_f:
            f = max_f
            
        derivative_beta = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_beta(alpha_matrix, beta_matrix, model, prefix, x)
        beta_matrix[d][f] += learning_rate * derivative_beta
        
        beta_matrix[d][f] = max(0, min(1, beta_matrix[d][f]))
        
        derivative_alpha = 1 / self.blend(alpha_matrix, beta_matrix, prefix, x, model, n) * self.derivative_of_P_to_alpha(alpha_matrix, beta_matrix, model, prefix, x)
        alpha_matrix[d][f] += learning_rate * derivative_alpha
        
        alpha_matrix[d][f] = max(-beta_matrix[d][f], alpha_matrix[d][f])
        
        
    
    def compute_alpha_beta(self, alpha_matrix, beta_matrix, prefix):
        max_f = len(alpha_matrix[0])
        d = len(prefix)
        f = len(set(prefix))
        
        if f > max_f:
            f = max_f
        
        return (alpha_matrix[d][f], beta_matrix[d][f])

In [4]:
# test build model and train
alpha_matrix = [[0.5, 0], [0.5, 0], [0.5, 0], [0.5, 0], [0.5, 0]]
beta_matrix = [[0.75, 0], [0.75, 0], [0.75, 0], [0.75, 0], [0.75, 0]]

trainer_1 = Blending_with_linear_param()
n_gram_model_1 = trainer_1.build_model_with_train(alpha_matrix, beta_matrix, "words_manual_en.txt", 4)

# verify
cand = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [chr(i) for i in range(ord('a'), ord('z') + 1)] + ["$"]

# alpha_matrix = [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
# beta_matrix = [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
def verify(trainer, prefix, model, n=4):
    s = 0
    for c in cand:
        prob = trainer.blend(alpha_matrix, beta_matrix, prefix, c, model)
        s += prob
        
    return s

print(verify(trainer_1, "ralv", n_gram_model_1))
print(verify(trainer_1, "^All", n_gram_model_1))
print(verify(trainer_1, "Trum", n_gram_model_1))
print(verify(trainer_1, "Russ", n_gram_model_1))
print(verify(trainer_1, "Koch", n_gram_model_1))
print(verify(trainer_1, "Rack", n_gram_model_1))

1.0
1.0000000000000004
1.0
1.0
1.0
1.0


In [3]:
# test build model and train
alpha_matrix = [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]
beta_matrix = [[0.75, 0.75, 0.75, 0.75, 0.75, 0.75], [0.75, 0.75, 0.75, 0.75, 0.75, 0.75], [0.75, 0.75, 0.75, 0.75, 0.75, 0.75], [0.75, 0.75, 0.75, 0.75, 0.75, 0.75], [0.75, 0.75, 0.75, 0.75, 0.75, 0.75]]

trainer_2 = Blending_with_2d_matrix()
n_gram_model_2 = trainer_2.build_model_with_train(alpha_matrix, beta_matrix, "words_manual_en.txt", 4)

# verify
cand = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [chr(i) for i in range(ord('a'), ord('z') + 1)] + ["$"]

def verify(trainer, prefix, model, n=4):
    s = 0
    for c in cand:
        prob = trainer.blend(alpha_matrix, beta_matrix, prefix, c, model)
        s += prob
        
    return s

print(verify(trainer_2, "ralv", n_gram_model_2))
print(verify(trainer_2, "^All", n_gram_model_2))
print(verify(trainer_2, "Trum", n_gram_model_2))
print(verify(trainer_2, "Russ", n_gram_model_2))
print(verify(trainer_2, "Koch", n_gram_model_2))
print(verify(trainer_2, "Rack", n_gram_model_2))

1.0
0.9999999999999896
1.0
0.9999999999994359
1.0
1.0


In [4]:
alpha_matrix

[[0.47800298113638034, 0.5, 0.5, 0.5, 0.5, 0.5],
 [0.5, 0.0693369271729633, 0.5, 0.5, 0.5, 0.5],
 [0.5, 0.3779192188812493, -1.5900221451692142e-05, 0.5, 0.5, 0.5],
 [0.5, 0.45874650241120857, 0.05099735198599677, 0, 0.5, 0.5],
 [0.5, 0.4458847059206506, 0.360093002532607, 0.0012161018566216645, 0, 0.5]]

In [25]:
# verify
alpha_matrix = [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]
beta_matrix = [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]


n_gram_model = n_gram_model_2
cand = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [chr(i) for i in range(ord('a'), ord('z') + 1)] + ["$"]

def verify(prefix, model, n=4):
    s = 0
    for c in cand:
        prob = blend(alpha_matrix, beta_matrix, prefix, c, model)
        s += prob
        
    return s

print(verify("ralv", n_gram_model))
print(verify("^All", n_gram_model))
print(verify("Trum", n_gram_model))
print(verify("Russ", n_gram_model))
print(verify("Koch", n_gram_model))
print(verify("Rack", n_gram_model))

0.9999461523162747
0.9999813837460723
0.9984558384814151
0.9998254423072425
0.9994184450157699
0.9998428132668576


In [ ]:
n_gram_model = build_model("words_manual_en.txt", 4)
count = 0
with open ("checked_words.txt", "r") as f :
    for word in f:
        score = word_prob (word, n_gram_model)
        blend_score = word_prob_blend (word, n_gram_model)
        print(f"word: {word}, original_score: {score}, blend_score: {blend_score}")
        
        count += 1
        if count == 100:
            break


In [ ]:
# check derivative

a1 = derivative_of_P_to_alpha(0.5, 0.75, n_gram_model, "epri", "m")
a2 = derivative_of_P_to_alpha(0.5, 0.75, n_gram_model, "epri", "x")
a3 = derivative_of_P_to_alpha(0.5, 0.75, n_gram_model, "epri", "f")

print(a1, a2, a3)

b1 = derivative_of_P_to_alpha(0.5, 0.75, n_gram_model, "Acke", "l")
b2 = derivative_of_P_to_alpha(0.5, 0.75, n_gram_model, "Acke", "x")
b3 = derivative_of_P_to_alpha(0.5, 0.75, n_gram_model, "Acke", "f")

print(b1, b2, b3)

c1 = derivative_of_P_to_beta(0.5, 0.75, n_gram_model, "epri", "m")
c2 = derivative_of_P_to_beta(0.5, 0.75, n_gram_model, "epri", "x")
c3 = derivative_of_P_to_beta(0.5, 0.75, n_gram_model, "epri", "f")

print(c1, c2, c3)

d1 = derivative_of_P_to_beta(0.5, 0.75, n_gram_model, "Acke", "l")
d2 = derivative_of_P_to_beta(0.5, 0.75, n_gram_model, "Acke", "x")
d3 = derivative_of_P_to_beta(0.5, 0.75, n_gram_model, "Acke", "f")

print(d1, d2, d3)

In [ ]:
alpha_matrix = [[], [], [], [], [1, 0]]
beta_matrix = [[], [], [], [], [1, 0]]

update_param(alpha_matrix, beta_matrix, "Acke", "l", n_gram_model)

print(alpha_matrix)
print(beta_matrix)

In [ ]:
# unused code
def build_model(self, word_file, n):
        """
        Build n-gram model of the words in  words_lang.txt
        """
        # read in all n+1 grams
        n_plus_1_gram_counts = {}
        with open(word_file, "r") as f:
            for word in f:
                if not word or word[0] == "#" or " " in word:
                    continue
                word = "^" * n + word.strip() + "$"
                for i in range(len(word) - n):
                    n_plus_1_gram = word[i:i + n + 1]
                    if n_plus_1_gram not in n_plus_1_gram_counts:
                        n_plus_1_gram_counts[n_plus_1_gram] = 1
                    else:
                        n_plus_1_gram_counts[n_plus_1_gram] += 1
        # with open ("tmpppp", "w") as f :
        #  for key in sorted(n_plus_1_gram_counts) :
        #    print (str({key: n_plus_1_gram_counts[key]})[1:-1], file=f)

        # Find conditional probability of n+1st from previous n-gram
        counts = {}
        n_grams = {x: {} for x in range(0, n + 1)}
        for N in range(n, -1, -1):
            n_gram = None
            for ng in sorted(n_plus_1_gram_counts):
                next_n = ng[:-1]
                if n_gram != next_n:
                    if n_gram == None:
                        n_gram = next_n
                    else:
                        s = sum([counts[x] for x in counts])
                        if not n_gram.startswith('^^'):  # at most one ^ at the start
                            n_grams[N][n_gram] = {c: counts[c] for c in counts}

                        n_gram = next_n
                        counts = {}

                counts[ng[-1]] = n_plus_1_gram_counts[ng]

            # process last case.  Should we put a sentinel in n_plus_1_gram_counts?
            s = sum([counts[x] for x in counts])
            n_grams[N][n_gram] = {c: counts[c] for c in counts}

            if N > 0:
                new_n = {}
                for key in n_plus_1_gram_counts:
                    suff = key[1:]
                    if suff in new_n:
                        new_n[suff] += 1
                    else:
                        new_n[suff] = 1
                n_plus_1_gram_counts = new_n

        # compress to variable n?
        # print(n_grams)

        for N in range(n):
            n_grams[N + 1].update(n_grams[N])

        return n_grams[n]
    
def word_prob(self, word, model):
    word = "^" + word + "$"
    pos = 1  # char after ^
    log_likelihood = 0
    # print ("   ", end="")
    while pos < len(word):
        done = False
        for i in range(pos):
            history = word[i:pos]
            if history in model:
                if history not in model:
                    # Should penalize this
                    history = history.lower()
                try:
                    log_likelihood += math.log(model[history][word[pos]] / sum(model[history].values()))
                except:
                    low = word[pos].lower()
                    if low in model[history]:
                        # Should penalize this
                        log_likelihood += math.log(model[history][low] / sum(model[history].values()))
                    else:
                        # pdb.set_trace ()
                        log_likelihood += -20
                # print (int(log_likelihood), end = " ")
                log_likelihood -= 3 * i  # penalize shorter histories
                done = True
                pos += 1
                break

        if not done:  # Transisition so unlikely, it was never seen
            # Should use "smoothing", but this will disappear
            # when the variable-length prefixes are implemented
            log_likelihood += -20
            # print (int(log_likelihood), end = " ")
            pos += 1

    # print()

    return log_likelihood / (len(word) - 1)  # didn't guess "^"
